In [1]:

import pandas as pd
import json
from algorithm import improve_system
from dotenv import load_dotenv
load_dotenv()


True

#### At this point, we've run some examples through our LLM graph. If not, refer to this post. 

#### We have a graph, and we can start to analyze the outputs.

In [2]:
with open("/Users/ankushgarg/Desktop/MIDS/epic-data-lab/validate-LLM-graphs/reddit-comments/graph.json", 'r') as json_file:
    graph_json = json.load(json_file)
    
edges = graph_json['edges']


In [3]:
from middleware import convert_edges_to_dag

dag = convert_edges_to_dag(edges=edges)


#### Read in dataset with your prompts and LLM responses

In [4]:
llm_responses = pd.read_excel("/Users/ankushgarg/Desktop/MIDS/epic-data-lab/validate-LLM-graphs/reddit-comments/reddit-comments-eval.xlsx")
llm_responses = llm_responses.head(1)
llm_responses

,error,post_id,prompt-1,prompt-2,prompt-3,prompt-4,input-post,input-replies,output-keywords,output-article_search,output-replies,output-reply,keywords-node,tool-node,tone-summary-node,reply-node,notes
0,NaN,10w0v9z,Extract two most important keywords from the i...,"Based on these keywords: ['Bill Gates', 'clima...","Your goal is to extract a tone (single), senti...",Your job is to write a reddit reply (in less t...,Bill Gates on why he’ll carry on using private...,"['Defund the oligarchs.', ""Even if I was a com...","[""['Bill Gates', 'climate change']""]",[{'url': 'https://www.gatesnotes.com/What-you-...,"{'tone': 'critical', 'sentiment': 'negative', ...",['While I understand the desire for public fig...,1.0,1.0,1.0,1.0,NaN


#### Use LLM-as-a-Judge to run the responses for validation

In [5]:
from llm_as_judge import run_validation_using_LLM

node_input_output_mappings = {
    "extract_keywords_from_title": ("prompt-1", "output-keywords"),
    "tool_search": ("prompt-2", "output-article_search"),
    "summarize_tone_sentiment_of_replies": ("prompt-3", "output-replies"),
    "generate_reply": ( "prompt-4", "output-reply")
}

llm_response_df = run_validation_using_LLM(
    dataframe=llm_responses,
    node_input_output_mappings=node_input_output_mappings
    )

/Users/ankushgarg/Desktop/MIDS/epic-data-lab/validate-LLM-graphs/.llm_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
# from scipe import convert_to_dag, run_validation_using_llm, find_problematic_node
# # "Systematic Chain Improvement and Problem Evaluation"

# # We have the graph from langgraph
# graph = Langgraph()
# # Middleware to convert it to a format that we need
# # People can write this however
# dag = convert_to_dag(graph) # We'll need this later

# data_of_inputs_outputs = pd.DataFrame()

# llm_responses_df = run_validation_using_llm(data_of_inputs_outputs)

# node = find_problematic_node(llm_responses_df)

# print(node)

# "Node: 1 is problematic"
# "It has 33% chance of failing on it's own, compared to 9% chance of failing"
# "because of node(n-1). Please focus on fixing Node 1."

# ## What are some ways node 1 can be fixed. 
# ## Could we analyze it?


In [21]:
improve_system(
    downstream_node='generate_reply',
    data=outputs,
    graph=dag
    )

Analyzing node: generate_reply
Overall failure probability for this node: 0.0000
Independent failure probability: 0.0000
Node failure because dep fails: 0
Conditional failure probabilities given upstream dependency failures:
  P(generate_reply fails | summarize_tone_sentiment_of_replies fails): 0.0000

Analyzing node: summarize_tone_sentiment_of_replies
Overall failure probability for this node: 0.0000
Independent failure probability: 0.0000
Node failure because dep fails: 0
Conditional failure probabilities given upstream dependency failures:
  P(summarize_tone_sentiment_of_replies fails | tool_search fails): 0.0000

Analyzing node: tool_search
Overall failure probability for this node: 0.0000
Independent failure probability: 0.0000
Node failure because dep fails: 0
Conditional failure probabilities given upstream dependency failures:
  P(tool_search fails | extract_keywords_from_title fails): 0.0000

Analyzing node: extract_keywords_from_title
Overall failure probability for this nod

/Users/ankushgarg/Desktop/MIDS/epic-data-lab/validate-LLM-graphs/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()
/Users/ankushgarg/Desktop/MIDS/epic-data-lab/validate-LLM-graphs/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()
/Users/ankushgarg/Desktop/MIDS/epic-data-lab/validate-LLM-graphs/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()


ValueError: max() arg is an empty sequence

In [ ]:
llm_response_df